### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.large-data :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojure.java.io :as io]
            [clojure.data.csv :as csv]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

### Initialise portfolio （Go back here everytime you want to restart.）

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(load-large-dataset "../../data-sorted-cleaned/data-CRSP-sorted-cleaned.csv" "main")
;; load compustat
(load-large-dataset "../../data-sorted-cleaned/data-Compustat-sorted-cleaned.csv" "compustat")
(set-main "main")
(init-portfolio "1963-02-05" 10)

"1963-02-05"

In [3]:
;; (next-date)
;; (keys (deref available-tics) )
;; (get-date)
;; (get-line)

In [9]:
(count (deref available-tics))
;; (end-order)

2038

In [4]:
(order-lazy "PPL" 10 :print true)

{"PPL" {:date "1963-02-05", :expiration 3, :quantity 10, :remaining false, :leverage true, :print true, :direct true}}

In [5]:
(time (while 
    (< (compare (get-date) "1965-02") 0)
          ;(order-lazy "PPL" 10 :direct false)
          (next-day)))

true
Order: 1963-02-06 | PPL | 10.000000.
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
tru

nil

In [11]:
(time (while 
    (< (compare (curr-date) "2016-02") 0)(next-day)))

"Elapsed time: 916551.674745 msecs"


nil

In [9]:
;; (take 20 (deref portfolio-value)) 
;; (view-portfolio-record 20)

### Write a strategy

The following code implements a trading strategy called Golden Rule:

MA 50 cross above the MA 200 (golden cross)

MA 200 cross below the MA 50 (death cross)

So in the codes, MA50 and MA200 are compared on a daily basis, if golden cross occurs, then you set a buy order; if death cross occurs, then you set a sell order first 



In [ ]:
(time (do (def MA50-vec-aapl [])
          (def MA200-vec-aapl [])
          (def MA50-vec-f [])
          (def MA200-vec-f [])
          (while (not= (get-date) "2016-12-29")
            (do
    ;; write your trading strategy here
              (def tics (deref available-tics)) ;20 ms
              (def MA50-vec-aapl (get-prev-n-days :PRC 50 "AAPL" MA50-vec-aapl (get (get tics "AAPL"):reference)))
              (def MA200-vec-aapl (get-prev-n-days :PRC 200 "AAPL" MA200-vec-aapl (get (get tics "AAPL") :reference)))
              (def MA50-vec-f (get-prev-n-days :PRC 50 "F" MA50-vec-f (get (get tics "F"):reference)))
              (def MA200-vec-f (get-prev-n-days :PRC 200 "F" MA200-vec-f (get (get tics "F") :reference)))
              (let [[MA50 MA200] [(moving-average :PRC MA50-vec-aapl) (moving-average :PRC MA200-vec-aapl)]]
                (if (> MA50 MA200)
                  (order "AAPL" 1 :reference (get (get tics "AAPL") :reference) :print false) 
                  (order "AAPL" 0 :remaining true :reference (get (get tics "AAPL") :reference))))
              (let [[MA50 MA200] [(moving-average :PRC MA50-vec-f) (moving-average :PRC MA200-vec-f)]]
                (if (> MA50 MA200)
                  (order "F" 1 :reference (get (get tics "F") :reference) :print false) 
                  (order "F" 0 :remaining true :reference (get (get tics "F") :reference))))
              ;(update-eval-report (get-date))
              (next-date)))))
(.close wrtr)

In [ ]:
(count (deref order-record))


### Check portfolio record

In [ ]:
;; view final portfolio
(view-portfolio)

In [12]:
;; view portfolio value and return
(view-portfolio-record 10)


|      :date | :tot-value | :daily-ret | :tot-ret |    :loan | :leverage |
|------------+------------+------------+----------+----------+-----------|
| 1963-02-05 |        $10 |      0.00% |    0.00% |    $0.00 |     0.00% |
| 1963-02-06 |        $11 |     -5.47% |   -5.47% | $-362.50 |   -96.95% |
| 1963-02-07 |         $7 |      7.16% |   12.10% | $-362.50 |   -97.97% |
| 1963-02-08 |         $6 |     -4.89% |   19.40% | $-362.50 |   -98.28% |
| 1963-02-11 |         $8 |      3.22% |    5.75% | $-362.50 |   -97.64% |
| 1963-02-12 |         $6 |     -0.00% |   19.96% | $-362.50 |   -98.30% |
| 1963-02-13 |         $7 |     -8.42% |   11.54% | $-362.50 |   -97.93% |
| 1963-02-14 |         $9 |      2.89% |    1.28% | $-362.50 |   -97.38% |
| 1963-02-15 |        $12 |     -5.12% |   -7.66% | $-362.50 |   -96.78% |
| 1963-02-18 |         $9 |     -0.00% |    2.08% | $-362.50 |   -97.42% |


nil

### Generate evaluation report

In [11]:
(update-eval-report (get-date))
(eval-report)

Execution error (NullPointerException) at clojure-backtesting.evaluate/eval-report (evaluate.clj:148).
null


class java.lang.NullPointerException: 

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)